In [1]:
from env.core import FunctionExpr, Param
from impl.goal_env import GoalEnv
from impl.node_types import HaveDefinition, AndNode, OrNode, IntBooleanNode

params = (Param.from_int(1), Param.from_int(2), Param.from_int(3))
p1, p2, p3 = params
goal = HaveDefinition(
    FunctionExpr.with_child(
        OrNode(
            AndNode(p1, p2, IntBooleanNode(1)),
            AndNode(p2, p3),
        ),
    )
)

env = GoalEnv(goal)

env.to_symbolic()

TypeError: Cannot create a consistent method resolution
order (MRO) for bases INode, Generic, IInstantiable